In [16]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import conbqa
import dimod
import matplotlib.pyplot as plt
from neal import SimulatedAnnealingSampler
import pandas as pd
from tensorflow.keras import layers, models


In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

In [18]:
# Dataset directories
train_dir = 'Dataset/train'
test_dir = 'Dataset/test'

# Parameters
img_height, img_width = 48, 48
batch_size = 32
num_classes = 7

In [19]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for testing set
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [20]:

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)


Found 28965 images belonging to 7 classes.
Found 7361 images belonging to 7 classes.


In [21]:
def NN_eval(n1, n2, n3, n4):

    model = models.Sequential([
        layers.Conv2D(int(n1), (3, 3), activation='relu', input_shape=(img_height, img_width, 1)),
#         BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.3),
        layers.Conv2D(int(n2), (3, 3), activation='relu'),
#         BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.3),
        layers.Conv2D(int(n3), (3, 3), activation='relu'),
#         BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(int(n4), activation='relu'),
#         layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model for fewer epochs to save computation time
    model.fit(train_generator, epochs=5, validation_data=test_generator, verbose=0)
    
    # Evaluate validation accuracy
    val_loss, val_accuracy = model.evaluate(test_generator, verbose=1)
    predictions = model.predict(test_generator)
    loss, accuracy = model.evaluate(test_generator)
    print(f"Test accuracy: {accuracy:.2f}")
    return 1 - val_accuracy

In [22]:
num_init = 5
#Random initial values for the bb function parameters
init_X = np.stack([
    np.linspace(32, 200, num_init)[np.random.permutation(num_init)]
    for _ in range(4)
], axis=1)
init_X


#Calculate init_Y
init_Y = np.zeros(num_init)
for i in range(len(init_X)):
  print("hello")
  init_Y[i] = NN_eval(init_X[i,0],init_X[i,1],init_X[i,2],init_X[i,3])

hello


2024-11-28 19:00:42.364892: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:32: Filling up shuffle buffer (this may take a while): 4 of 8
2024-11-28 19:00:52.682987: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:32: Filling up shuffle buffer (this may take a while): 6 of 8
2024-11-28 19:01:00.818946: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


231/231 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.2863 - loss: 1.7759
231/231 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.2929 - loss: 1.7730
Test accuracy: 0.29
hello
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.2876 - loss: 1.7966
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.2857 - loss: 1.7972
Test accuracy: 0.28
hello
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.2890 - loss: 1.8025
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.2824 - loss: 1.8023
Test accuracy: 0.29
hello
231/231 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.3016 - loss: 1.7641
231/231 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.3108 - loss: 1.7609
Test accuracy: 0.31
hello
231/231 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.3476 - loss: 1.7012
231/231 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step

In [23]:

model = conbqa.CONBQA(init_X, init_Y, np.array([6, 6, 6, 6]), np.array([1025, 1025, 1025, 1025]), maximization=False)

model.set_encoding_method_to_RandomSubspaceCoding(60, v=20.0)
model.set_learning_model_to_linear()

solver = SimulatedAnnealingSampler()

In [ ]:
nepoch = 5
for i in range(nepoch):
    print("hi")
    model.generate_hyperrectangles_and_encode()
    model.learn_weights()
    q = model.convert_maximization_of_learned_function_into_qubo()
    bqm = dimod.BQM.from_qubo(q)
    res = solver.sample(bqm)
    new_z = res.record['sample'][0]
    new_x1, new_x2, new_x3, new_x4 = model.decode(new_z)
    new_y = NN_eval(new_x1, new_x2, new_x3, new_x4)
    new_x_combined = np.concatenate([new_x1.flatten(), new_x2.flatten(), new_x3.flatten(), new_x4.flatten()])
    model.add_data(new_x_combined, new_y)
    # progress bar
    print("\r", "=" * int((i + 1) / nepoch * 50), ">", "." * (50 - int((i + 1) / nepoch * 50)), end="", sep="")

hi


/Users/manasa_vasupalli/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


231/231 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.3495 - loss: 1.7131
231/231 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.3415 - loss: 1.7103
Test accuracy: 0.34
==========>........................................hi


/Users/manasa_vasupalli/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


231/231 ━━━━━━━━━━━━━━━━━━━━ 17s 74ms/step - accuracy: 0.3574 - loss: 1.6528
231/231 ━━━━━━━━━━━━━━━━━━━━ 17s 74ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 17s 74ms/step - accuracy: 0.3560 - loss: 1.6568
Test accuracy: 0.36
====================>..............................hi


/Users/manasa_vasupalli/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


231/231 ━━━━━━━━━━━━━━━━━━━━ 1051s 5s/step - accuracy: 0.3443 - loss: 1.6843
231/231 ━━━━━━━━━━━━━━━━━━━━ 13s 57ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 953s 4s/step - accuracy: 0.3455 - loss: 1.6896
Test accuracy: 0.35
==============================>....................hi


/Users/manasa_vasupalli/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [80]:
best_Y = model.y.min()
best_Y
argmin_Y = np.argmin(model.y)
argmin_Y
#Finding best X
model.X[argmin_Y]

array([ 12.08288525, 219.43691636, 136.07514848, 239.50900548])

In [83]:
accuracy = NN_eval(12,219,136,239)

231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.3945 - loss: 1.5578
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4097 - loss: 1.5364
Test accuracy: 0.40
